In [ ]:
# exploring clinical variables

In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import collections
import asyncio
import getpass
import re
from datetime import datetime as dt
import os,sys,re
import urllib3
import prettytable
from collections import Counter
import seaborn as sns
import random

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
import os
cwd= os.getcwd()

time: 1.15 ms


In [3]:
cwd

'/Users/geickelb1'

time: 3.28 ms


In [ ]:
#importing clinical variable csv
lab = f'data/interim/{today}-winemag_priceGBP.csv'



labs_all48=  pd.read_csv('/Documents/GitHub/mimiciii-antibiotics-modeling/data/wrangled/labs_all48.csv', index_col=0)
bg_all48=  pd.read_csv('/Documents/GitHub/mimiciii-antibiotics-modeling/data/wrangled/labs_all48.csv', index_col=0)
vital_all48=  pd.read_csv('/Documents/GitHub/mimiciii-antibiotics-modeling/data/wrangled/vital_all48.csv', index_col=0)
